# Step 0: Setup
Configure headers with API key.

In [10]:
import os
from dotenv import load_dotenv

load_dotenv()
CLIMATE_ENGINE_API_KEY = os.environ.get('CLIMATE_ENGINE_API_KEY')

HEADERS = {
    'Accept': 'application/json',
    'Authorization': CLIMATE_ENGINE_API_KEY
}

# Step 1: API Exploration

Test the API to infer details regarding how it works and what it accepts.

### Fetch Subcatchment Geometries

Load the GeoJSON file containing subcatchment polygons with TopazID identifiers.

In [11]:
import pandas as pd
import requests

# URL for the subcatchment GeoJSON
GEOJSON_URL = 'https://wc-prod.bearhive.duckdns.org/weppcloud/runs/batch;;nasa-roses-2025;;wa-0/disturbed9002_wbt/download/dem/wbt/subcatchments.WGS.geojson'

# Fetch and parse the GeoJSON data
resp = requests.get(GEOJSON_URL)
if resp.ok:
    geojson_data = resp.json()
    print(f"Loaded {len(geojson_data['features'])} subcatchments")
    
    # Extract unique TopazIDs and their geometries
    subcatchments = []
    for feature in geojson_data['features']:
        topaz_id = feature['properties']['TopazID']
        geometry = feature['geometry']
        subcatchments.append({'topaz_id': topaz_id, 'geometry': geometry})
    
    # Create DataFrame and get unique subcatchments
    subcatchments_df = pd.DataFrame(subcatchments)
    unique_subcatchments = subcatchments_df.groupby('topaz_id').first().reset_index()
    print(f"Found {len(unique_subcatchments)} unique TopazIDs")
    print(f"Sample TopazIDs: {unique_subcatchments['topaz_id'].head().tolist()}")
else:
    print(f'Error {resp.status_code}: {resp.text}')

Loaded 555 subcatchments
Found 392 unique TopazIDs
Sample TopazIDs: [32, 33, 42, 43, 52]


### Check for Disjointed Subcatchments & Test MultiPolygon Support

Analyze the data to see if any TopazIDs have multiple polygons, and test if the API supports MultiPolygon geometries.

*Spoiler Alert: [It does](https://support.climateengine.org/article/152-formatting-coordinates-for-api-requests), but a separate value is returned for each segment of the MultiPolygon.*

In [12]:
API_URL = 'https://api.climateengine.org'

# Check for duplicate TopazIDs (indicating disjointed subcatchments)
topaz_counts = subcatchments_df['topaz_id'].value_counts()
disjointed = topaz_counts[topaz_counts > 1]

print(f"Analysis of TopazID distribution:")
print(f"Total features: {len(subcatchments_df)}")
print(f"Unique TopazIDs: {len(topaz_counts)}")
print(f"Disjointed subcatchments (TopazID with multiple polygons): {len(disjointed)}")

if len(disjointed) > 0:
    print(f"\nTop 10 TopazIDs with most polygons:")
    print(disjointed.head(10))
    
    # Test API behavior with multiple polygons
    print("\n" + "="*60)
    print("Testing API behavior with multiple polygons...")
    print("="*60)
    
    # Get a TopazID with multiple polygons
    test_topaz = disjointed.index[0]
    test_geometries = subcatchments_df[subcatchments_df['topaz_id'] == test_topaz]['geometry'].tolist()
    
    print(f"\nTest TopazID: {test_topaz}")
    print(f"Number of polygons: {len(test_geometries)}")
    
    # Test API call with multiple polygons
    url = f'{API_URL}/timeseries/native/coordinates'
    
    # Format: list of outer rings for each polygon
    # According to docs: [[polygon1_coords], [polygon2_coords], ...]
    # Docs: https://support.climateengine.org/article/152-formatting-coordinates-for-api-requests
    all_polygon_coords = []
    for geom in test_geometries:
        if geom['type'] == 'Polygon':
            all_polygon_coords.append(geom['coordinates'][0])  # Outer ring only
    
    params = {
        'dataset': 'OPENET_CONUS',
        'variable': 'et_ensemble_mad',
        'start_date': '2020-01-01',
        'end_date': '2020-01-31',  # Just one month for testing
        'coordinates': str(all_polygon_coords),
        'area_reducer': 'mean'
    }
    
    try:
        resp = requests.get(url, headers=HEADERS, params=params, timeout=60)
        if resp.ok:
            result = resp.json()
            print("✓ SUCCESS: API accepts multiple polygons!")
            print(f"  Response status: {resp.status_code}")
            
            # Check how the API returns data for multiple polygons
            num_data_items = len(result.get('Data', []))
            print(f"  Number of data items returned: {num_data_items}")
            print(f"  Number of polygons sent: {len(all_polygon_coords)}")
            
            if num_data_items == len(all_polygon_coords):
                print("\n✓ API returns SEPARATE data for each polygon")
                print("  → Need to aggregate results across polygons for disjointed subcatchments")
                print("\n  Sample data structure:")
                import json
                print(json.dumps(result['Data'][0], indent=2)[:500] + "...")
            elif num_data_items == 1:
                print("\n✓ API returns AGGREGATED data across all polygons")
                print("  → No additional aggregation needed")
            else:
                print(f"\n⚠ Unexpected: {num_data_items} data items for {len(all_polygon_coords)} polygons")
        else:
            print(f"✗ FAILED: API returned error {resp.status_code}")
            print(f"  Error message: {resp.text}")
    except Exception as e:
        print(f"✗ EXCEPTION: {e}")
        print("  Could not determine API behavior")
else:
    print("\n✓ No disjointed subcatchments found - all TopazIDs have single polygons")
    print("  No need to handle multiple polygons per subcatchment")

Analysis of TopazID distribution:
Total features: 555
Unique TopazIDs: 392
Disjointed subcatchments (TopazID with multiple polygons): 89

Top 10 TopazIDs with most polygons:
topaz_id
1343    10
1623     8
1223     7
1672     7
152      5
1433     5
1452     5
1392     5
73       5
1693     4
Name: count, dtype: int64

Testing API behavior with multiple polygons...

Test TopazID: 1343
Number of polygons: 10
✓ SUCCESS: API accepts multiple polygons!
  Response status: 200
  Number of data items returned: 10
  Number of polygons sent: 10

✓ API returns SEPARATE data for each polygon
  → Need to aggregate results across polygons for disjointed subcatchments

  Sample data structure:
{
  "Metadata": {
    "DRI_OBJECTID": "[[-123.71241681201863, 47.30913865947637], [-123.71241318476312, 47.30886873208591], [-123.71201631836027, 47.30887119844715], [-123.71201269317766, 47.30860127102055], [-123.71161582874234, 47.308603735984484], [-123.71160133661905, 47.307524026055646], [-123.711998192973

### Check for Polygons with Holes (Interior Rings)

Analyze whether any subcatchments have interior rings (holes) and test if the API supports them.

In [13]:
# Check if any polygons have holes (interior rings)
# According to GeoJSON spec: coordinates[0] = outer ring, coordinates[1+] = holes

polygons_with_holes = []
for idx, row in subcatchments_df.iterrows():
    geom = row['geometry']
    if geom['type'] == 'Polygon':
        # Polygon has holes if len(coordinates) > 1
        num_rings = len(geom['coordinates'])
        if num_rings > 1:
            polygons_with_holes.append({
                'topaz_id': row['topaz_id'],
                'num_holes': num_rings - 1,
                'geometry': geom
            })

print(f"Analysis of interior rings (holes):")
print(f"Total polygon features: {len(subcatchments_df[subcatchments_df['geometry'].apply(lambda g: g['type'] == 'Polygon')])}")
print(f"Polygons with holes: {len(polygons_with_holes)}")

if len(polygons_with_holes) > 0:
    print(f"\nSample polygons with holes:")
    for item in polygons_with_holes[:5]:
        print(f"  TopazID {item['topaz_id']}: {item['num_holes']} hole(s)")
    
    # Test if API handles holes correctly
    print("\n" + "="*60)
    print("Testing API behavior with polygon holes...")
    print("="*60)
    
    test_poly = polygons_with_holes[0]
    print(f"\nTest TopazID: {test_poly['topaz_id']}")
    print(f"Number of holes: {test_poly['num_holes']}")
    print(f"Number of coordinate rings: {len(test_poly['geometry']['coordinates'])}")
    
    # Test 1: Pass only outer ring (current approach)
    url = f'{API_URL}/timeseries/native/coordinates'
    outer_ring_only = [test_poly['geometry']['coordinates'][0]]
    
    params1 = {
        'dataset': 'OPENET_CONUS',
        'variable': 'et_ensemble_mad',
        'start_date': '2020-01-01',
        'end_date': '2020-01-31',
        'coordinates': str(outer_ring_only),
        'area_reducer': 'mean'
    }
    
    print("\nTest 1: Passing OUTER RING ONLY...")
    try:
        resp1 = requests.get(url, headers=HEADERS, params=params1, timeout=60)
        if resp1.ok:
            result1 = resp1.json()
            print(f"  ✓ Success - got {len(result1.get('Data', []))} data items")
            if 'Data' in result1 and len(result1['Data']) > 0 and 'Data' in result1['Data'][0]:
                sample_value = result1['Data'][0]['Data'][0].get('et_ensemble_mad (mm)', 'N/A')
                print(f"  Sample ET value: {sample_value}")
        else:
            print(f"  ✗ Failed: {resp1.status_code}")
    except Exception as e:
        print(f"  ✗ Exception: {e}")
    
    # Test 2: Pass full polygon with holes
    print("\nTest 2: Passing FULL POLYGON (with holes)...")
    # According to GeoJSON, a polygon with holes is: [[outer], [hole1], [hole2], ...]
    # But Climate Engine API expects a list of coordinate rings
    # Let's try passing all rings
    all_rings = test_poly['geometry']['coordinates']
    
    params2 = {
        'dataset': 'OPENET_CONUS',
        'variable': 'et_ensemble_mad',
        'start_date': '2020-01-01',
        'end_date': '2020-01-31',
        'coordinates': str(all_rings),
        'area_reducer': 'mean'
    }
    
    try:
        resp2 = requests.get(url, headers=HEADERS, params=params2, timeout=60)
        if resp2.ok:
            result2 = resp2.json()
            print(f"  ✓ Success - got {len(result2.get('Data', []))} data items")
            if 'Data' in result2 and len(result2['Data']) > 0 and 'Data' in result2['Data'][0]:
                sample_value = result2['Data'][0]['Data'][0].get('et_ensemble_mad (mm)', 'N/A')
                print(f"  Sample ET value: {sample_value}")
            
            # Compare results
            if resp1.ok and 'Data' in result1 and 'Data' in result2:
                val1 = result1['Data'][0]['Data'][0].get('et_ensemble_mad (mm)', 0)
                val2 = result2['Data'][0]['Data'][0].get('et_ensemble_mad (mm)', 0)
                print(f"\n  Comparison:")
                print(f"    Outer ring only: {val1}")
                print(f"    With holes:      {val2}")
                if abs(val1 - val2) < 0.01:
                    print(f"    → Values are SAME - API may be ignoring holes")
                else:
                    print(f"    → Values DIFFER - API may be treating each ring separately")
        else:
            print(f"  ✗ Failed: {resp2.status_code} - {resp2.text[:200]}")
            print(f"    → API may not support polygons with holes in this format")
    except Exception as e:
        print(f"  ✗ Exception: {e}")
    
    print("\n" + "="*60)
    print("CONCLUSION:")
    if not resp2.ok:
        print("⚠ API likely does NOT support holes - use outer ring only")
        print("  This means ET may be OVERESTIMATED for areas with holes")
        print("  (calculating ET over the hole areas too)")
    elif resp1.ok and abs(val1 - val2) < 0.01:
        print("⚠ API appears to IGNORE interior rings")
        print("  This means ET may be OVERESTIMATED for areas with holes")
    else:
        print("✓ API may handle holes correctly")
        print("  Need to pass full polygon coordinates including holes")
    print("="*60)
else:
    print("\n✓ No polygons with holes found - all polygons are simple")
    print("  No need to worry about interior rings")

Analysis of interior rings (holes):
Total polygon features: 555
Polygons with holes: 3

Sample polygons with holes:
  TopazID 1673: 4 hole(s)
  TopazID 843: 1 hole(s)
  TopazID 763: 1 hole(s)

Testing API behavior with polygon holes...

Test TopazID: 1673
Number of holes: 4
Number of coordinate rings: 5

Test 1: Passing OUTER RING ONLY...
  ✓ Success - got 1 data items
  Sample ET value: 3.7311

Test 2: Passing FULL POLYGON (with holes)...
  ✓ Success - got 1 data items
  Sample ET value: 3.7311

Test 2: Passing FULL POLYGON (with holes)...
  ✓ Success - got 5 data items
  Sample ET value: 3.7311

  Comparison:
    Outer ring only: 3.7311
    With holes:      3.7311
    → Values are SAME - API may be ignoring holes

CONCLUSION:
⚠ API appears to IGNORE interior rings
  This means ET may be OVERESTIMATED for areas with holes
  ✓ Success - got 5 data items
  Sample ET value: 3.7311

  Comparison:
    Outer ring only: 3.7311
    With holes:      3.7311
    → Values are SAME - API may be ig

### Merge Disjointed Subcatchments

If disjointed subcatchments exist, create a proper merged geometry for each TopazID that combines all polygons.

In [14]:
def merge_polygon_geometries(geometries):
    """
    Merge multiple Polygon geometries into a unified structure for API calls.
    Returns a dict with 'type' and 'coordinates' that can be passed to fetch_et_timeseries.
    
    For multiple polygons, coordinates will be a list of outer rings.
    Note: Interior rings (holes) are currently ignored since the Climate Engine API
    likely doesn't support them properly. This may cause slight overestimation of ET
    for polygons with holes.
    """
    if len(geometries) == 1:
        return geometries[0]
    
    # Collect all polygon outer rings (ignoring holes for now)
    all_outer_rings = []
    for geom in geometries:
        if geom['type'] == 'Polygon':
            # coordinates[0] is the outer ring (per GeoJSON spec RFC 7946)
            # coordinates[1+] would be holes, which we skip
            all_outer_rings.append(geom['coordinates'][0])
        elif geom['type'] == 'MultiPolygon':
            # Flatten MultiPolygon into separate outer rings
            for polygon_coords in geom['coordinates']:
                # polygon_coords[0] is the outer ring of this polygon
                all_outer_rings.append(polygon_coords[0])
    
    # Return as a special format that our fetch function can handle
    return {
        'type': 'MultiPolygon',
        'coordinates': all_outer_rings  # List of outer rings only
    }

# Create properly merged subcatchments
print("Merging disjointed subcatchments...")

# Group geometries by topaz_id and merge them manually to avoid pandas dict expansion issues
merged_data = []
for topaz_id, group in subcatchments_df.groupby('topaz_id'):
    geometries = group['geometry'].tolist()
    merged_geom = merge_polygon_geometries(geometries)
    merged_data.append({'topaz_id': topaz_id, 'geometry': merged_geom})

unique_subcatchments = pd.DataFrame(merged_data)

print(f"Processed {len(unique_subcatchments)} unique TopazIDs")

# Show statistics
geometry_types = unique_subcatchments['geometry'].apply(lambda g: g['type']).value_counts()
print(f"\nGeometry type distribution:")
print(geometry_types)

if 'MultiPolygon' in geometry_types:
    multi_count = geometry_types['MultiPolygon']
    print(f"\n⚠ {multi_count} TopazIDs have disjointed subcatchments (multiple polygons)")
    print("  These will be handled by passing all polygon coordinates to the API")
    print("  and aggregating the results appropriately.")

Merging disjointed subcatchments...
Processed 392 unique TopazIDs

Geometry type distribution:
geometry
Polygon         303
MultiPolygon     89
Name: count, dtype: int64

⚠ 89 TopazIDs have disjointed subcatchments (multiple polygons)
  These will be handled by passing all polygon coordinates to the API
  and aggregating the results appropriately.


# Step 2: Get Available OpenET Variables/Models

Query the Climate Engine API to find all available ET variables for the OPENET_CONUS dataset.

In [15]:
import json

API_URL = 'https://api.climateengine.org'

# Get available variables for OPENET_CONUS dataset
metadata_url = f'{API_URL}/metadata/dataset_variables'
params = {'dataset': 'OPENET_CONUS'}

resp = requests.get(metadata_url, headers=HEADERS, params=params)
if resp.ok:
    variables_data = resp.json()
    print(json.dumps(variables_data, indent=2))
    
    # Extract ET model variables
    if 'Variables' in variables_data:
        et_variables = variables_data['Variables']
        print(f"\nFound {len(et_variables)} variables")
else:
    print(f'Error {resp.status_code}: {resp.text}')

{
  "Request": {
    "dataset": "OPENET_CONUS",
    "export_format": null,
    "coll_url": "https://support.climateengine.org/article/78-openet"
  },
  "Data": {
    "variables": [
      "et_eemetric",
      "et_sims",
      "et_ssebop",
      "et_geesebal",
      "et_ptjpl",
      "et_disalexi",
      "et_ensemble_mad",
      "et_ensemble_mad_min",
      "et_ensemble_mad_max"
    ],
    "variable names": [
      "Evapotranspiration eeMETRIC",
      "Evapotranspiration SIMS",
      "Evapotranspiration SSEBop",
      "Evapotranspiration geeSEBAL",
      "Evapotranspiration PT-JPL",
      "Evapotranspiration DisALEXI",
      "Evapotranspiration Ensemble Median",
      "Evapotranspiration Ensemble Minimum",
      "Evapotranspiration Ensemble Maximum"
    ]
  }
}


## Step 3: Fetch ET Timeseries Data

Create a function to fetch monthly ET data for each subcatchment polygon using the Climate Engine timeseries API.

In [16]:
from datetime import datetime
import time
import numpy as np
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Create a session with retry configuration
def create_session_with_retries(retries=3, backoff_factor=1, status_forcelist=(500, 502, 504)):
    """
    Create a requests session with automatic retry logic.
    
    Parameters:
    - retries: Number of retry attempts (default: 3)
    - backoff_factor: Exponential backoff factor (default: 1 -> 1s, 2s, 4s)
    - status_forcelist: HTTP status codes to retry on
    
    Returns:
    - Configured requests.Session object
    """
    session = requests.Session()
    retry_strategy = Retry(
        total=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
        allowed_methods=["GET", "POST"],  # Retry on GET and POST
        raise_on_status=False  # Don't raise exception on bad status
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

# Create session once to reuse across requests
api_session = create_session_with_retries(retries=3, backoff_factor=1)

def fetch_et_timeseries(geometry, variable, start_date, end_date, area_reducer='mean'):
    """
    Fetch ET timeseries data for a polygon or multipolygon geometry.
    
    Parameters:
    - geometry: GeoJSON geometry object (Polygon or MultiPolygon)
    - variable: ET variable name (e.g., 'et_ensemble_mad')
    - start_date: Start date (YYYY-MM-DD format)
    - end_date: End date (YYYY-MM-DD format)
    - area_reducer: Spatial aggregation method (default: 'mean')
    
    Returns:
    - Dictionary with:
        - 'data': Processed API response with timeseries data, or None if request fails
        - 'nodata_stats': {'total_entries': int, 'nodata_count': int}
    
    Note: For MultiPolygon, the API returns separate data for each polygon.
    We aggregate by taking the mean across all polygons for each date.
    NoData values (-9999.0) are filtered during aggregation.
    Uses automatic retry with exponential backoff via requests session.
    """
    url = f'{API_URL}/timeseries/native/coordinates'
    
    # Extract coordinates from geometry
    if geometry['type'] == 'Polygon':
        coordinates = [geometry['coordinates'][0]]  # Outer ring only
    elif geometry['type'] == 'MultiPolygon':
        # For disjointed subcatchments: pass all polygon outer rings
        coordinates = geometry['coordinates']  # Already formatted as list of outer rings
    else:
        raise ValueError(f"Unsupported geometry type: {geometry['type']}")
    
    params = {
        'dataset': 'OPENET_CONUS',
        'variable': variable,
        'start_date': start_date,
        'end_date': end_date,
        'coordinates': str(coordinates),
        'area_reducer': area_reducer
    }
    
    try:
        resp = api_session.get(url, headers=HEADERS, params=params, timeout=60)
        if resp.ok:
            raw_response = resp.json()
            
            # Count NoData in raw response BEFORE aggregation
            variable_key = f"{variable} (mm)"
            total_entries = 0
            nodata_count = 0
            
            if 'Data' in raw_response:
                for polygon_data in raw_response['Data']:
                    if 'Data' in polygon_data:
                        for entry in polygon_data['Data']:
                            if variable_key in entry:
                                total_entries += 1
                                if entry[variable_key] == -9999.0:
                                    nodata_count += 1
            
            # If multiple polygons, aggregate the results (filters NoData)
            if geometry['type'] == 'MultiPolygon' and len(raw_response.get('Data', [])) > 1:
                processed_response = aggregate_multipolygon_results(raw_response, variable)
            else:
                processed_response = raw_response
            
            return {
                'data': processed_response,
                'nodata_stats': {
                    'total_entries': total_entries,
                    'nodata_count': nodata_count
                }
            }
        else:
            print(f'Error {resp.status_code}: {resp.text[:200]}')
            return None
    except Exception as e:
        print(f'Exception: {e}')
        return None

def aggregate_multipolygon_results(api_response, variable):
    """
    Aggregate results from multiple polygons into a single timeseries.
    Takes the mean across all polygons for each date.
    Filters out NoData values (-9999.0) before aggregation.
    """
    if 'Data' not in api_response or len(api_response['Data']) <= 1:
        return api_response
    
    # Collect all timeseries data from each polygon
    all_polygon_data = []
    for polygon_result in api_response['Data']:
        if 'Data' in polygon_result:
            all_polygon_data.append(polygon_result['Data'])
    
    if not all_polygon_data:
        return api_response
    
    # Aggregate by date - take mean across all polygons
    variable_key = f"{variable} (mm)"
    date_values = {}
    
    for polygon_timeseries in all_polygon_data:
        for entry in polygon_timeseries:
            if 'Date' in entry and variable_key in entry:
                date = entry['Date']
                value = entry[variable_key]
                # Filter out NoData values (-9999.0)
                if value != -9999.0:
                    if date not in date_values:
                        date_values[date] = []
                    date_values[date].append(value)
    
    # Create aggregated timeseries
    aggregated_data = []
    for date, values in sorted(date_values.items()):
        if values:  # Only include dates with valid data
            aggregated_data.append({
                'Date': date,
                variable_key: np.mean(values)  # Mean across all valid polygons
            })
    
    # Return in same format as API response
    return {
        'Data': [{
            'Data': aggregated_data
        }]
    }

# Test with first subcatchment
test_subcatchment = unique_subcatchments.iloc[0]
print(f"Testing with TopazID: {test_subcatchment['topaz_id']}")
print(f"Geometry type: {test_subcatchment['geometry']['type']}")

test_result = fetch_et_timeseries(
    geometry=test_subcatchment['geometry'],
    variable='et_ensemble_mad',
    start_date='2020-01-01',
    end_date='2020-12-31'
)

if test_result and test_result['data']:
    print(f"\nAPI Response structure:")
    print(json.dumps(test_result['data'], indent=1)[:1000] + "...")
    print(f"\nNoData Stats: {test_result['nodata_stats']}")
else:
    print("\n⚠ Test request failed - check your network connection")

Testing with TopazID: 32
Geometry type: MultiPolygon

API Response structure:
{
 "Data": [
  {
   "Data": [
    {
     "Date": "2020-01-01",
     "et_ensemble_mad (mm)": 4.09895
    },
    {
     "Date": "2020-02-01",
     "et_ensemble_mad (mm)": 17.7427
    },
    {
     "Date": "2020-03-01",
     "et_ensemble_mad (mm)": 37.2716
    },
    {
     "Date": "2020-04-01",
     "et_ensemble_mad (mm)": 78.5382
    },
    {
     "Date": "2020-05-01",
     "et_ensemble_mad (mm)": 108.2616
    },
    {
     "Date": "2020-06-01",
     "et_ensemble_mad (mm)": 106.68719999999999
    },
    {
     "Date": "2020-07-01",
     "et_ensemble_mad (mm)": 121.7559
    },
    {
     "Date": "2020-08-01",
     "et_ensemble_mad (mm)": 116.9302
    },
    {
     "Date": "2020-09-01",
     "et_ensemble_mad (mm)": 73.34215
    },
    {
     "Date": "2020-10-01",
     "et_ensemble_mad (mm)": 29.85235
    },
    {
     "Date": "2020-11-01",
     "et_ensemble_mad (mm)": 9.31655
    },
    {
     "Date": "2020-12-0

## Step 4: Process All Subcatchments and Create Final Table

Fetch ET data for all subcatchments and multiple models, then format into the requested table structure.

**NoData Handling:** The API may return -9999.0 as a NoData indicator. These values are automatically filtered out during processing.

**Important:** NoData tracking happens on the **raw API response** before any MultiPolygon aggregation. This ensures accurate counting even when:
- All polygons in a MultiPolygon return NoData
- Some polygons have valid data while others have NoData
- The aggregation process filters out NoData values

In [17]:
def process_timeseries_response(response_data, topaz_id, variable):
    """
    Parse API response and extract year, month, value data.
    Filters out NoData values (-9999.0) from the results.
    
    Returns:
    - List of dictionaries with topaz_id, year, month, model, value
    - NoData values are excluded from the output
    """
    results = []
    
    if not response_data or 'Data' not in response_data:
        return results
    
    # The API returns data in a nested structure
    # response_data['Data'] is a list with one element per coordinate/polygon
    # Each element has a 'Data' field with the actual timeseries
    data_list = response_data['Data']
    
    for coordinate_data in data_list:
        if 'Data' not in coordinate_data:
            continue
            
        # Extract the timeseries data
        timeseries = coordinate_data['Data']
        
        # Create the variable key (e.g., 'et_ensemble_mad (mm)')
        variable_key = f"{variable} (mm)"
        
        for entry in timeseries:
            if 'Date' in entry and variable_key in entry:
                # Parse date
                date_str = entry['Date']
                try:
                    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                    year = date_obj.year
                    month = date_obj.month
                    value = entry[variable_key]
                    
                    # Filter out NoData values (-9999.0)
                    if value == -9999.0:
                        continue
                    
                    results.append({
                        'topaz_id': topaz_id,
                        'year': year,
                        'month': month,
                        'model': variable,
                        'value': value
                    })
                except Exception as e:
                    print(f"Error parsing entry {entry}: {e}")
    
    return results

from tqdm.notebook import tqdm
from IPython.display import display, clear_output

# Configuration
START_YEAR = 2020
END_YEAR = 2023
START_DATE = f'{START_YEAR}-01-01'
END_DATE = f'{END_YEAR}-12-31'

# ET variables to fetch (you can add more from the metadata query)
ET_VARIABLES = [
    'et_ensemble_mad',
    # 'et_disalexi',
    # 'et_eemetric',
    # 'et_geesebal',
    # 'et_pt_jpl',
    # 'et_sims',
    # 'et_ssebop'
]

print(f"Processing {len(unique_subcatchments)} subcatchments")
print(f"Fetching {len(ET_VARIABLES)} variables")
print(f"Date range: {START_DATE} to {END_DATE}")
print(f"="*60)

# Collect all data
all_data = []

# Toggle between test mode and full processing
TEST_MODE = False  # Set to True for testing, False to process all
test_limit = 5     # Number of subcatchments to test with

# Select subcatchments to process
if TEST_MODE:
    subcatchments_to_process = unique_subcatchments.head(test_limit)
    total = test_limit
    print(f"⚠ TEST MODE: Processing only {test_limit} subcatchments")
else:
    subcatchments_to_process = unique_subcatchments
    total = len(unique_subcatchments)
    print(f"Processing all {total} subcatchments")

print(f"="*60)

# Stats tracking
failed_requests = []
successful_requests = 0
total_requests = total * len(ET_VARIABLES)
nodata_by_subcatchment = {}  # Track NoData occurrences

# Main processing loop with progress bar
for idx, row in tqdm(subcatchments_to_process.iterrows(), total=len(subcatchments_to_process), 
                      desc="Overall Progress", unit="subcatchment"):
    topaz_id = row['topaz_id']
    geometry = row['geometry']
    
    for variable in ET_VARIABLES:
        # Fetch data (NoData tracking happens inside the function)
        result = fetch_et_timeseries(
            geometry=geometry,
            variable=variable,
            start_date=START_DATE,
            end_date=END_DATE
        )
        
        if result and result['data']:
            # Extract NoData statistics
            nodata_count = result['nodata_stats']['nodata_count']
            total_entries = result['nodata_stats']['total_entries']
            
            # Parse the processed response
            parsed_data = process_timeseries_response(result['data'], topaz_id, variable)
            
            # Track NoData statistics
            if nodata_count > 0:
                if topaz_id not in nodata_by_subcatchment:
                    nodata_by_subcatchment[topaz_id] = {'total': 0, 'nodata': 0}
                nodata_by_subcatchment[topaz_id]['total'] += total_entries
                nodata_by_subcatchment[topaz_id]['nodata'] += nodata_count
            
            all_data.extend(parsed_data)
            successful_requests += 1
        else:
            failed_requests.append({'topaz_id': topaz_id, 'variable': variable})
        
        # Be nice to the API
        time.sleep(0.5)

# Create final DataFrame
final_df = pd.DataFrame(all_data)

print(f"\n{'='*60}")
print(f"PROCESSING COMPLETE")
print(f"{'='*60}")
print(f"Total API requests: {total_requests}")
print(f"Successful: {successful_requests}")
print(f"Failed: {len(failed_requests)}")
print(f"Success rate: {100*successful_requests/total_requests:.1f}%")

if failed_requests:
    print(f"\n⚠ Warning: {len(failed_requests)} requests failed:")
    for item in failed_requests[:5]:
        print(f"  - TopazID {item['topaz_id']}, variable {item['variable']}")
    if len(failed_requests) > 5:
        print(f"  ... and {len(failed_requests)-5} more")

if nodata_by_subcatchment:
    total_nodata = sum(info['nodata'] for info in nodata_by_subcatchment.values())
    total_values = sum(info['total'] for info in nodata_by_subcatchment.values())
    nodata_rate = total_nodata / total_values * 100 if total_values > 0 else 0
    
    print(f"\nNoData Statistics:")
    print(f"  Subcatchments with NoData: {len(nodata_by_subcatchment)}")
    print(f"  Total NoData values filtered: {total_nodata:,}")
    print(f"  Overall NoData rate: {nodata_rate:.2f}%")
    
    # Show worst cases
    worst_cases = sorted(
        [(tid, info['nodata']/info['total']*100) for tid, info in nodata_by_subcatchment.items()],
        key=lambda x: x[1],
        reverse=True
    )[:5]
    
    if worst_cases:
        print(f"\n  Top 5 subcatchments by NoData rate:")
        for tid, rate in worst_cases:
            print(f"    - TopazID {tid}: {rate:.1f}%")

print(f"\nFinal Dataset Summary:")
print(f"  Total records: {len(final_df):,}")
print(f"  Unique subcatchments: {final_df['topaz_id'].nunique()}")
print(f"  Unique models: {final_df['model'].nunique()}")
print(f"  Date range: {final_df['year'].min()}-{final_df['year'].max()}")

print(f"\nFirst 10 rows:")
display(final_df.head(10))

Processing 392 subcatchments
Fetching 1 variables
Date range: 2020-01-01 to 2023-12-31
Processing all 392 subcatchments


Overall Progress:   0%|          | 0/392 [00:00<?, ?subcatchment/s]


PROCESSING COMPLETE
Total API requests: 392
Successful: 392
Failed: 0
Success rate: 100.0%

NoData Statistics:
  Subcatchments with NoData: 179
  Total NoData values filtered: 407
  Overall NoData rate: 2.68%

  Top 5 subcatchments by NoData rate:
    - TopazID 1173: 7.3%
    - TopazID 1392: 7.1%
    - TopazID 1693: 6.8%
    - TopazID 42: 6.2%
    - TopazID 883: 6.2%

Final Dataset Summary:
  Total records: 18,635
  Unique subcatchments: 392
  Unique models: 1
  Date range: 2020-2023

First 10 rows:


,topaz_id,year,month,model,value
0,32,2020,1,et_ensemble_mad,4.09895
1,32,2020,2,et_ensemble_mad,17.74270
2,32,2020,3,et_ensemble_mad,37.27160
3,32,2020,4,et_ensemble_mad,78.53820
4,32,2020,5,et_ensemble_mad,108.26160
5,32,2020,6,et_ensemble_mad,106.68720
6,32,2020,7,et_ensemble_mad,121.75590
7,32,2020,8,et_ensemble_mad,116.93020
8,32,2020,9,et_ensemble_mad,73.34215
9,32,2020,10,et_ensemble_mad,29.85235


## Step 5: Save and Analyze Results

Display summary statistics and save the final table to a Parquet file.

In [18]:
# Display sample of final table with all columns
print("Final Table Structure:")
print(f"Columns: {final_df.columns.tolist()}")
print(f"\nSample data (topaz_id, year, month, model, value):")
display(final_df[['topaz_id', 'year', 'month', 'model', 'value']].head(20))

# Summary statistics by model
print("\n\nSummary by Model:")
summary = final_df.groupby('model')['value'].agg(['count', 'mean', 'min', 'max', 'std'])
display(summary)

# Summary by year
print("\n\nSummary by Year:")
yearly = final_df.groupby('year')['value'].agg(['count', 'mean'])
display(yearly)

# Save to data directory
os.makedirs('data', exist_ok=True)

# Save to Parquet (more efficient than CSV)
output_file = 'data/et_data_by_subcatchment.parquet'
final_df.to_parquet(output_file, index=False, engine='pyarrow', compression='snappy')

# Also save metadata
import json

# Calculate NoData statistics for metadata
nodata_stats = {}
if nodata_by_subcatchment:
    total_nodata = sum(info['nodata'] for info in nodata_by_subcatchment.values())
    total_values = sum(info['total'] for info in nodata_by_subcatchment.values())
    nodata_stats = {
        'subcatchments_with_nodata': len(nodata_by_subcatchment),
        'total_nodata_filtered': total_nodata,
        'total_values_examined': total_values,
        'nodata_rate_percent': round(total_nodata / total_values * 100, 2) if total_values > 0 else 0
    }

metadata = {
    'date_created': pd.Timestamp.now().isoformat(),
    'num_records': len(final_df),
    'num_subcatchments': final_df['topaz_id'].nunique(),
    'models': final_df['model'].unique().tolist(),
    'date_range': {
        'start': START_DATE,
        'end': END_DATE
    },
    'years': sorted(final_df['year'].unique().tolist()),
    'data_quality': {
        'nodata_filtering': 'Enabled - values of -9999.0 excluded',
        **nodata_stats
    }
}

metadata_file = 'data/et_data_by_subcatchment_metadata.json'
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\n{'='*60}")
print(f"DATA SAVED SUCCESSFULLY")
print(f"{'='*60}")
print(f"Data file: {output_file}")
print(f"Metadata file: {metadata_file}")
print(f"Total records: {len(final_df):,}")

# Show file size
file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
print(f"File size: {file_size_mb:.2f} MB")

print(f"\nTo read the data later:")
print(f"  df = pd.read_parquet('{output_file}')")

Final Table Structure:
Columns: ['topaz_id', 'year', 'month', 'model', 'value']

Sample data (topaz_id, year, month, model, value):


,topaz_id,year,month,model,value
0,32,2020,1,et_ensemble_mad,4.09895
1,32,2020,2,et_ensemble_mad,17.74270
2,32,2020,3,et_ensemble_mad,37.27160
3,32,2020,4,et_ensemble_mad,78.53820
4,32,2020,5,et_ensemble_mad,108.26160
5,32,2020,6,et_ensemble_mad,106.68720
6,32,2020,7,et_ensemble_mad,121.75590
7,32,2020,8,et_ensemble_mad,116.93020
8,32,2020,9,et_ensemble_mad,73.34215
9,32,2020,10,et_ensemble_mad,29.85235




Summary by Model:


,count,mean,min,max,std
model,,,,,
et_ensemble_mad,18635,58.523257,0.0,167.0,46.315797




Summary by Year:


,count,mean
year,,
2020,4704,56.422447
2021,4646,61.426900
2022,4704,52.459184
2023,4581,63.962523



DATA SAVED SUCCESSFULLY
Data file: data/et_data_by_subcatchment.parquet
Metadata file: data/et_data_by_subcatchment_metadata.json
Total records: 18,635
File size: 0.13 MB

To read the data later:
  df = pd.read_parquet('data/et_data_by_subcatchment.parquet')


## Notes

### Handling NoData Values:
The Climate Engine API may return **-9999.0** as a NoData indicator for certain dates/locations. The script automatically handles this:
- **Filtering**: All -9999.0 values are excluded from the output
- **Aggregation**: For MultiPolygon subcatchments, NoData values are filtered before computing means
- **Tracking**: The script tracks and reports how many NoData values were encountered
- **Metadata**: NoData statistics are saved to the metadata file for quality assessment

**Impact**: If a subcatchment has high NoData rates, it may indicate:
- The location is outside the OpenET coverage area
- Data is not available for the requested time period
- The geometry may need verification

To diagnose NoData issues in detail, run: `python diagnose_nodata.py`

### Handling Disjointed Subcatchments:
The script automatically detects and handles subcatchments that consist of multiple disjointed polygons (same TopazID, multiple features). According to the Climate Engine API documentation:
- The API accepts multiple polygons in a single request as: `[[polygon1_coords], [polygon2_coords], ...]`
- The API returns **separate** data for each polygon
- The script aggregates results by computing the **mean** across all polygons for each date
- This approach assumes all polygon parts of a subcatchment should be weighted equally

### Handling Polygons with Holes (Interior Rings):
According to the GeoJSON specification (RFC 7946), polygons can have interior rings (holes):
- `coordinates[0]` = exterior ring (outer boundary)
- `coordinates[1], coordinates[2], ...` = interior rings (holes)

**Current Implementation:**
- The script currently uses **only the outer ring** of each polygon
- Interior rings (holes) are **ignored**
- This may cause **slight overestimation** of ET values for polygons with holes (calculating ET over the hole areas)

The Climate Engine API documentation doesn't clearly specify support for polygon holes, and testing (Step 1) helps determine if the API properly handles them. If your data has polygons with holes and accurate ET calculation is critical, you may need to contact Climate Engine support for clarification on hole handling

### To process all subcatchments:
1. In Step 4, change `TEST_MODE = False` to `True`
2. Or modify the subcatchment selection logic as needed
3. Processing all subcatchments may take considerable time depending on the number

### To add more ET models:
Uncomment additional variables in the `ET_VARIABLES` list in Step 4. Available variables from OpenET include:
- `et_ensemble_mad` - Ensemble median absolute deviation
- `et_disalexi` - DisALEXI model
- `et_eemetric` - eeMETRIC model
- `et_geesebal` - geeSEBAL model  
- `et_pt_jpl` - PT-JPL model
- `et_sims` - SIMS model
- `et_ssebop` - SSEBop model

### API Rate Limiting:
The code includes a 0.5 second delay between API calls. Adjust the `time.sleep()` value if you encounter rate limiting errors.